## Import labraries

In [1]:
import pandas as pd
import numpy as np

import jellyfish

import seaborn as sns

## Import database with clean sectors name and raw routes' name

In [2]:
data_sector_clean = pd.read_csv('routes_info_sector_clean.csv')
data_sector_clean.head()

,grade,name,name_clean,sector,sector_clean,ascents,fos_ratio,recommendations,stars
0,7a,Freedom is a Battle,NaN,Trebenna West,Trebenna West,781,66 %,10 %,3
1,7a,Karınca,NaN,Magara,Magara,699,55 %,16 %,4
2,7b,Lycian Highway,NaN,Trebenna West,Trebenna West,645,70 %,13 %,4
3,6b+,Saxafon,NaN,Sarkit,Sarkit,591,66 %,20 %,4
4,6b,Nirvana,NaN,Magara,Magara,589,78 %,10 %,3


## Dataset analysis

Because of troubles with parsing, data has lots of duplicates. We will solve this problem later.

In [3]:
print('Count of duplicates:', data_sector_clean.duplicated().sum())

Count of duplicates: 173


In [4]:
data_sector_clean.drop_duplicates(inplace=True)

In [5]:
data_sector_clean

,grade,name,name_clean,sector,sector_clean,ascents,fos_ratio,recommendations,stars
0,7a,Freedom is a Battle,NaN,Trebenna West,Trebenna West,781,66 %,10 %,3
1,7a,Karınca,NaN,Magara,Magara,699,55 %,16 %,4
2,7b,Lycian Highway,NaN,Trebenna West,Trebenna West,645,70 %,13 %,4
3,6b+,Saxafon,NaN,Sarkit,Sarkit,591,66 %,20 %,4
4,6b,Nirvana,NaN,Magara,Magara,589,78 %,10 %,3
...,...,...,...,...,...,...,...,...,...
2100,6a+,Session of Hapiness,NaN,Anatolia Sag,Anatolia,1,0 %,0 %,0
2101,6b+,Comftably,NaN,Echoes,Echoes,1,100 %,0 %,0
2102,6a,En Iyisi o (Neco),NaN,Poseidon,Poseidon,1,100 %,0 %,0
2103,7c+,Sunatorium,NaN,Cesme,Cesme,1,0 %,0 %,4


In [6]:
# change name_clean type to object

data_sector_clean['name_clean'] = data_sector_clean['name_clean'].astype('object')

Info about sectors.

In [7]:
(
    data_sector_clean
    .groupby(['sector_clean'])['name']
    .count()
    .sort_values(ascending=False)
)

sector_clean
Unknown Sector      336
Trebenna West       249
Alabalik            115
Sarkit              107
Küllüin              79
                   ... 
de que fas aqui?      1
güsel                 1
Külüin Terras         1
Akyalar               1
okuzini cave          1
Name: name, Length: 65, dtype: int64

We can see Unknown Sector. Let's try to recover their names.

In [8]:
name_with_several_sectors = (
    data_sector_clean
    .groupby(['name'])['sector_clean']
    .count()
    .sort_values(ascending=False)
)

name_with_several_sectors = (
    name_with_several_sectors[name_with_several_sectors > 1]
)

New routes without names usually are down written like ‘n.n.’. But some time later they are given normal name with no rewriting old info. So I prefer to delete them.

In [9]:
print('Routes without name: ')
(data_sector_clean['name'] == 'N.N.').sum()

Routes without name: 


72

In [10]:
# deleting

data_sector_clean.drop(data_sector_clean.loc[data_sector_clean['name']=='N.N.'].index, inplace=True)

In [11]:
print('Routes without name: ')
(data_sector_clean['name'] == 'N.N.').sum()

Routes without name: 


0

Also, we may have routes without ascent. It’s not interesting info, ’cause we are trying to find the most popular routes.

In [12]:
data_sector_clean.drop(data_sector_clean.loc[data_sector_clean['ascents']==0].index, inplace=True)

After deleting we can work with routes with several sectors. Let's recover some unknown sectors.

In [13]:
# analyse one route

data_sector_clean[data_sector_clean['name']=='Triologie']

,grade,name,name_clean,sector,sector_clean,ascents,fos_ratio,recommendations,stars
50,6b+,Triologie,NaN,Trebenna West,Trebenna West,276,84 %,5 %,3
460,7a,Triologie,NaN,Trebene,Trebenna West,44,73 %,16 %,3
627,5b,Triologie,NaN,Trebenna West,Trebenna West,23,91 %,4 %,4
895,7a,Triologie,NaN,Trebenna West,Trebenna West,8,88 %,0 %,3
1218,6a,Triologie,NaN,Trebenna West 4,Trebenna West,3,67 %,33 %,0
1797,6b+,Triologie,NaN,Unknown Sector,Unknown Sector,1,100 %,0 %,3


As we can see, route Triologie has several grades. That the reason of several sectors name, but it still has UNKNOWN sector that can be recovered.

In [14]:
names_set = set(name_with_several_sectors.index.tolist()) - {'N.N.'}

In [15]:
for name_with_unknown in names_set:
    index_to_recover = (
        data_sector_clean[
            (data_sector_clean['name']==name_with_unknown) &
            (data_sector_clean['sector_clean']=='Unknown Sector')
        ].index.tolist()
    )

    index_with_right_info = (
        data_sector_clean[
            (data_sector_clean['name']==name_with_unknown) &
            (data_sector_clean['sector_clean']!='Unknown Sector')
            ].index.tolist()
    )
    
    if len(index_with_right_info) != 0:
        data_sector_clean.loc[index_to_recover, 'sector_clean'] = (
            data_sector_clean.loc[index_with_right_info[0], 'sector_clean']
        )

Checking result.

In [16]:
(
    data_sector_clean
    .groupby(['sector_clean'])['name']
    .count()
    .sort_values(ascending=False)
)

sector_clean
Trebenna West       260
Unknown Sector      219
Alabalik            121
Sarkit              111
Küllüin              83
                   ... 
de que fas aqui?      1
güsel                 1
Funky Chicken         1
Alcyone               1
okuzini cave          1
Name: name, Length: 62, dtype: int64

Were recovered about a hundred routes. Another routes better to delete, because we don't know their location. Also, as routes with no ascents.

In [17]:
data_sector_clean.drop(data_sector_clean.loc[data_sector_clean['ascents']==0].index, inplace=True)


## Grouping by sectors and clean names

Function for future routes' name correction.

In [18]:
def routes_name_revers (similar_pairs, column_name):
    """
    Function for changing wrong names in routes' name column.
    :param similar_pairs: list of tuples with similar names of routes
    :return: None
    """
    # Creating dictionary with 'wrong_value': ['true_value', ascents'_different]}
    
    # ATTENTION! Searching will be organized in pivot table
    # grouping by sectors, but cleaning will be realized in origin table 

    similar_pairs_dict = {}

    for pair_names in similar_pairs:
        x = pair_names[0]
        y = pair_names[1]

        x_ascents = (
            data_by_sector[data_by_sector[column_name] == x]['ascents']
            .sum())
        y_ascents = (
            data_by_sector[data_by_sector[column_name] == y]['ascents']
            .sum())

        difference = abs(x_ascents - y_ascents)

        # if x-name is more popular than y-name,
        # it has more ascents, and we will remember in dict
        # y-name as a key and x-name as right name

        if x_ascents > y_ascents:

            try:
                # if dict already has this sector as a key,
                # we should check more popular route name variation

                if difference > similar_pairs_dict[y][1]:
                    similar_pairs_dict[y] = [x, difference]
            except:
                # if not, just write new key and value

                similar_pairs_dict[y] = [x, difference]

        else:

            try:
                if difference > similar_pairs_dict[x][1]:
                    similar_pairs_dict[x] = [y, difference]
            except:
                similar_pairs_dict[x] = [y, difference]


        # cleaning-part of function

        for index, row in data_by_sector.iterrows():
            route_old_name = row[column_name]

            if route_old_name in similar_pairs_dict:
                data_sector_clean.loc[index, 'name_clean'] = similar_pairs_dict[route_old_name][0]
            else:
                data_sector_clean.loc[index, 'name_clean'] = route_old_name

Grouping by sectors.

In [19]:
for sector in data_sector_clean['sector_clean'].unique().tolist():
    data_by_sector = (
        data_sector_clean[data_sector_clean['sector_clean'] == sector]
    )

    # Create list with unique names.

    list_names = (data_by_sector['name'].sort_values().unique()).tolist()

    # Searching misspells. Using # levenshtein_distance — 2
    # creating cross-table with names and levin dist value
    
    df_names_similarity = (
        pd.DataFrame(columns=list_names, index=list_names, dtype='float')
    )
    
    # fill this cross-table

    for i in range(len(list_names)):
        for j in range(len(list_names)):
            if i >= j:
                df_names_similarity.iloc[i, j] = (
                    jellyfish.levenshtein_distance(
                        list_names[i].lower(),
                        list_names[j].lower())
                )
            else:
                df_names_similarity.iloc[i, j] = np.nan

    # Collecting similar pairs in tuple with levenshtein_distance value.

    LOW_EDGE = 2

    jellyfish_filter = (df_names_similarity.values <= LOW_EDGE) 
                        # & (df_names_similarity.values != 0))
    
    similar_pairs_leven_dist = [
        (
            df_names_similarity.index[x],
            df_names_similarity.columns[y],
            df_names_similarity.iloc[x, y]
        )
        for x, y in zip(*np.where(jellyfish_filter))
    ]

    # use function to find the most popular sectors name variation

    routes_name_revers(similar_pairs_leven_dist, 'name')

In [20]:
data_sector_clean.head()

,grade,name,name_clean,sector,sector_clean,ascents,fos_ratio,recommendations,stars
0,7a,Freedom is a Battle,Freedom is a Battle,Trebenna West,Trebenna West,781,66 %,10 %,3
1,7a,Karınca,Karınca,Magara,Magara,699,55 %,16 %,4
2,7b,Lycian Highway,Lycian Highway,Trebenna West,Trebenna West,645,70 %,13 %,4
3,6b+,Saxafon,Saxafon,Sarkit,Sarkit,591,66 %,20 %,4
4,6b,Nirvana,Nirvana,Magara,Magara,589,78 %,10 %,3


Let's check the result.

In [21]:
print(
    f"Routes before cleaning: {data_sector_clean['name'].nunique()}\n"
    f"Routes after cleaning: {data_sector_clean['name_clean'].nunique()}"
)

Routes before cleaning: 1665
Routes after cleaning: 1370


## Union duplicates

Transform percent's columns to numeric.

In [22]:
fos_ratio_count = (
        data_sector_clean['fos_ratio'].str.replace('%', '').astype(int) / 100 * data_sector_clean['ascents']
)

recom_count = (
        data_sector_clean['recommendations'].str.replace('%', '').astype(int) / 100 * data_sector_clean['ascents']
)


Paste info in new columns.

In [23]:
data_sector_clean.insert(loc=7, column='fos_ratio_count', value=fos_ratio_count)

data_sector_clean.insert(loc=9, column='recom_count', value=recom_count)

In [24]:
data_sector_clean.head()

,grade,name,name_clean,sector,sector_clean,ascents,fos_ratio,fos_ratio_count,recommendations,recom_count,stars
0,7a,Freedom is a Battle,Freedom is a Battle,Trebenna West,Trebenna West,781,66 %,515.46,10 %,78.10,3
1,7a,Karınca,Karınca,Magara,Magara,699,55 %,384.45,16 %,111.84,4
2,7b,Lycian Highway,Lycian Highway,Trebenna West,Trebenna West,645,70 %,451.50,13 %,83.85,4
3,6b+,Saxafon,Saxafon,Sarkit,Sarkit,591,66 %,390.06,20 %,118.20,4
4,6b,Nirvana,Nirvana,Magara,Magara,589,78 %,459.42,10 %,58.90,3


Create pivot table by sector, name and grade. Sum ascents and recalculate percent’s columns.

In [25]:
data_routes_union = (
    data_sector_clean
    .sort_values(by=['ascents'], ascending=False)
    .groupby(['sector_clean', 'name_clean', 'grade'])
    .agg(
        {
            'sector': 'first',
            'ascents': 'sum',
            'fos_ratio_count': 'sum',
            'recom_count': 'sum',
            'stars': 'first' 
        }
    )
    .reset_index()
)

In [26]:
data_routes_union

,sector_clean,name_clean,grade,sector,ascents,fos_ratio_count,recom_count,stars
0,Akdeniz,4X4,8b,Akdeniz,12,0.00,0.00,3
1,Akdeniz,Akdeniz,7b+,Akdeniz,13,4.03,1.95,4
2,Akdeniz,Atatürk,7a+,Akdeniz,2,1.00,0.00,3
3,Akdeniz,Bebe Biskuvi,6c,Akdeniz,34,23.12,1.02,3
4,Akdeniz,Django,7c+,Akdeniz,5,0.00,0.00,4
...,...,...,...,...,...,...,...,...
1511,new,hmmm1,6c,new,1,1.00,0.00,0
1512,new,hmmm1,6c+,new,1,1.00,0.00,0
1513,new,hmmm1,7a,new,1,0.00,0.00,0
1514,new,hmmm1,7b,new,1,0.00,0.00,0


## Recalc ration back to percents

In [27]:
data_routes_union['fos_ratio'] = data_routes_union.apply(lambda row: round(row['fos_ratio_count'] / row['ascents'], 2), axis=1)

data_routes_union['recommendation'] = data_routes_union.apply(lambda row: round(row['recom_count'] / row['ascents'], 2), axis=1)

In [28]:
data_routes_union

,sector_clean,name_clean,grade,sector,ascents,fos_ratio_count,recom_count,stars,fos_ratio,recommendation
0,Akdeniz,4X4,8b,Akdeniz,12,0.00,0.00,3,0.00,0.00
1,Akdeniz,Akdeniz,7b+,Akdeniz,13,4.03,1.95,4,0.31,0.15
2,Akdeniz,Atatürk,7a+,Akdeniz,2,1.00,0.00,3,0.50,0.00
3,Akdeniz,Bebe Biskuvi,6c,Akdeniz,34,23.12,1.02,3,0.68,0.03
4,Akdeniz,Django,7c+,Akdeniz,5,0.00,0.00,4,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
1511,new,hmmm1,6c,new,1,1.00,0.00,0,1.00,0.00
1512,new,hmmm1,6c+,new,1,1.00,0.00,0,1.00,0.00
1513,new,hmmm1,7a,new,1,0.00,0.00,0,0.00,0.00
1514,new,hmmm1,7b,new,1,0.00,0.00,0,0.00,0.00


In [29]:
data_routes_union = data_routes_union[[
    'grade', 
    'name_clean', 
    'sector',
    'sector_clean',
    'ascents',
    'fos_ratio',
    'recommendation',
    'stars'
]]

In [30]:
data_routes_union

,grade,name_clean,sector,sector_clean,ascents,fos_ratio,recommendation,stars
0,8b,4X4,Akdeniz,Akdeniz,12,0.00,0.00,3
1,7b+,Akdeniz,Akdeniz,Akdeniz,13,0.31,0.15,4
2,7a+,Atatürk,Akdeniz,Akdeniz,2,0.50,0.00,3
3,6c,Bebe Biskuvi,Akdeniz,Akdeniz,34,0.68,0.03,3
4,7c+,Django,Akdeniz,Akdeniz,5,0.00,0.00,4
...,...,...,...,...,...,...,...,...
1511,6c,hmmm1,new,new,1,1.00,0.00,0
1512,6c+,hmmm1,new,new,1,1.00,0.00,0
1513,7a,hmmm1,new,new,1,0.00,0.00,0
1514,7b,hmmm1,new,new,1,0.00,0.00,0


Check duplicates in name.

In [31]:
data_routes_union[data_routes_union[['name_clean', 'grade']].duplicated(keep=False)]

,grade,name_clean,sector,sector_clean,ascents,fos_ratio,recommendation,stars
244,6c+,Gekko Kebap,Deli Kasap,Deli Kasap,12,0.67,0.00,1
455,6c+,Gekko Kebap,Kebap,Kebap,35,0.49,0.09,3
515,7a,Atlas,Küllüin,Küllüin,9,0.89,0.11,3
546,6a+,No Reservations,Küllüin,Küllüin,41,0.95,0.03,3
571,7a,Atlas,Külüin Terras,Külüin Terras,35,0.71,0.00,3
575,7a,Camelita,Left Cave,Left Cave,21,0.62,0.14,3
650,7b,Blue Hotel,Mevlana,Mevlana,192,0.52,0.13,4
667,7b,Montenegro,Mevlana,Mevlana,83,0.30,0.17,3
676,6c+,Semazen,Mevlana,Mevlana,258,0.70,0.12,4
723,7b,Blue Hotel,Ottoman,Ottoman,16,0.38,0.25,4


Some duplicates in name because of not perfect cleaning in sectors' name.

In [32]:
data_routes_union = data_routes_union[data_routes_union['sector_clean'] != 'Unknown Sector']

In [33]:
data_routes_union.to_csv('routes_all_clean.csv', encoding='utf-8', index=False)